In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score,RocCurveDisplay,auc
from sklearn.model_selection import train_test_split, GridSearchCV,  LeavePGroupsOut, cross_validate, GroupKFold, StratifiedGroupKFold
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier,XGBRFClassifier
import matplotlib.pyplot as plt
import joblib
import random
import itertools
from scipy.stats import randint,uniform


from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV

In [ ]:
#--------------------------------------------------------------------------------------
#------ Import Data and create dataframe with all applicable features
#--------------------------------------------------------------------------------------
df=pd.read_csv("../data/51061fe4eb0cb9da61ebd13d39879eefb5bfe33f103d3a80c7123deb5e1cd8c9.dat", compression="gzip",sep=";")
#Data Preprocessing
headers = ["target__y","group__uid"]
for c in df.columns:
    if c[:5]=="feats":
        headers.append(c)
pdf = pd.DataFrame(df,columns=headers)


In [ ]:
#--------------------------------------------------------------------------------------
#------ Remove erronenous daga points
#--------------------------------------------------------------------------------------
exclude = ["target__y","group__uid","feats__cirk_vikt","feats__bw","feats__sex","feats__pnage_days"]
for column in pdf.columns.values:
    if column not in exclude:
        pdf = pdf[(pdf[column]!=-99999) & (pdf[column] != -999)]

In [ ]:
#--------------------------------------------------------------------------------------
#------ LabelEncoder
#--------------------------------------------------------------------------------------
le = LabelEncoder()
le.fit(pdf["group__uid"])
pdf["group__uid"] = le.transform(pdf["group__uid"])


In [ ]:
#--------------------------------------------------------------------------------------
#------ Standard Scaling
#--------------------------------------------------------------------------------------
scaler = StandardScaler()
to_scale = [c for c in headers if c not in ["target__y","group__uid"]]
pdf[to_scale] = scaler.fit_transform(pdf[to_scale])

In [ ]:
#--------------------------------------------------------------------------------------
#------ KNN Imputer
#--------------------------------------------------------------------------------------
print(f"Missing {pdf.isna().any(axis=1).sum()} out of {pdf.any(axis=1).sum()}")
imputer = KNNImputer()
imputer.fit(pdf)
pdf = pd.DataFrame(imputer.transform(pdf),columns=pdf.columns.values)
print(f"Missing {pdf.isna().any(axis=1).sum()} out of {pdf.any(axis=1).sum()}")

In [ ]:
#--------------------------------------------------------------------------------------
#------ Separate dataframe and calculate scale pos weight
#--------------------------------------------------------------------------------------
ids = pdf.pop("group__uid")
y = pdf.pop("target__y")
X = pdf
# print(ids)
# print(noPos,noNeg,scale_pos_weight)
#StratifiedGroupKFold
noPos = (y==1).sum()
noNeg = (y==0).sum()
scale_pos_weight=noNeg/noPos
# print(noPos,noNeg,scale_pos_weight)
#--------------------------------------------------------
#--------------------------------------------------------